In [1]:
import json
import os
import pandas as pd

In [2]:
metric = 'f1_score_weighted'

datasets = {
    'binary': [31, 37, 44, 1462, 1479, 1510, 40945],
    'multiclass': [23, 36, 54, 181, 1466, 40691, 40975],
    'multilabel_native': [285, 41464, 41465, 41468, 41470, 41471, 41473],
    'multilabel_powerset': ['285ps', '41464ps', '41465ps', '41468ps', '41470ps', '41471ps', '41473ps']
}

frameworks = [
    '4intelligence', 'autogluon', 'autokeras', 'autopytorch', 'autosklearn', 'evalml', 'fedot',
    'flaml', 'gama', 'h2o', 'lightautoml', 'lightwood', 'mljar', 'naive', 'pycaret', 'tpot',
]

In [3]:
all_results_list = [] # Results for all scenarios

for scenario, dataset_ids in datasets.items():
    
    results_list = []  # Results for this scenario

    for dataset_id in dataset_ids:

        for framework in frameworks:
    
            try:
                with open(f'results/{scenario}/{dataset_id}/automl_{framework}.json', 'r') as fp:
                    content = json.load(fp)
                    f1_scores = [x[metric] for x in content['results']]  # Extract scores
    
                    for trial_idx, score in enumerate(f1_scores):  # Store each trial
                        trial_result = {
                            "Dataset": dataset_id,
                            "Framework": framework,
                            "Trial": trial_idx + 1,  # Make trials 1-based for readability
                            "F1 Score": score
                        }
                        results_list.append(trial_result)
                        all_results_list.append(trial_result)
    
            except Exception as e:
                # Missing file or error → No results for this framework/dataset
                trial_result_error = {
                    "Dataset": dataset_id,
                    "Framework": framework,
                    "Trial": None,
                    "F1 Score": None
                }
                results_list.append(trial_result_error)
                all_results_list.append(trial_result_error)

    # Convert list of dicts to a DataFrame (for this scenario)
    filename = f"stats/{scenario}/f1_scores_results.csv"
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    df = pd.DataFrame(results_list)
    df.to_csv(filename, index=False)

# Convert list of dicts to a DataFrame (for all scenarios)
all_filename = "stats/all/f1_scores_results.csv"
os.makedirs(os.path.dirname(all_filename), exist_ok=True)
all_df = pd.DataFrame(all_results_list)
all_df.to_csv(all_filename, index=False)